In [1]:
import xml, os, pandas as pd, glob
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import fromstring, ElementTree
import requests, re
import time
import requests
import string
from bs4 import BeautifulSoup
from tqdm import tqdm
import random

In [2]:
os.chdir("C://Users//Ruben//Documents//GitHub//TheForeign//SCC//output-data")
npp_ts = pd.read_csv('npp_timespans.csv', sep = ';')

In [3]:
os.chdir("D://Scriptie//Data//csv-metadata")
list_csv = glob.glob('*.csv')

In [4]:
def GetTS(newspaper_title):
    sy = npp_ts.loc[npp_ts['npp'] == newspaper_title, 'sy'].iloc[0]
    ey = npp_ts.loc[npp_ts['npp'] == newspaper_title, 'ey'].iloc[0]
    t = ey - sy
    return t

In [5]:
def clean_and_split_str(txt):
    #strip_special_chars = re.compile("[^A-Za-z0-9#]+")
    translator = str.maketrans('', '', string.punctuation)
    txt = txt.translate(translator)
    txt = re.sub('\s+', ' ', txt).strip()
    txt = txt.lower()
    return txt

In [6]:
url1= "http://services.kb.nl/mdo/oai?verb=GetRecord&identifier=DDD:"
url3 = "&metadataPrefix=didl"

In [7]:
def DateXML(soup_element):
    t = str(soup_element.find_all("date")[0])[6:10]
    return t

In [8]:
def OCRXML(soup_element):
    t = soup_element.find_all("OCRConfidencelevel")
    t = [str(i) for i in t]
    t = [i.replace('<OCRConfidencelevel>', '') for i in t]
    t = [i.replace('<OCRConfidencelevel/>', '') for i in t]
    t = [i.replace('</OCRConfidencelevel>', '') for i in t]
    t = [i for i in t if i]
    t = [int(i.replace('0.', '')) for i in t]
    t = (sum(t) / len(t)) / 1000
    return t


In [9]:
def GetTitleNppYear(newspaper_title, year):
    fn = [fn for fn in list_csv if fn[0:4] == newspaper_title][0]
    
    df = pd.read_csv(fn, sep = '\t')
    df['id'] = df.id.str[:-6]
    df['date'] = df.date.str.slice(0,4)
    
    df = df.drop_duplicates(subset='id', keep="first")
    df = df[df['date'] == str(year)]
    df = df.sample(n=30)
    df = df.reset_index(drop=True)
    
    titles_list = list()
    
    for issue in list(df.id):
        url = url1 + issue + url3
        result = requests.get(url)
        content = result.content
        soup = BeautifulSoup(content, "xml")
        list_titles = soup.find_all("title")
        list_titles = [str(i) for i in list_titles]
        list_titles = [i.replace('<title>', '') for i in list_titles]
        list_titles = [i.replace('</title>', '') for i in list_titles]
        list_titles = [clean_and_split_str(i) for i in list_titles]

        for title in list_titles:
            t = [title, OCRXML(soup)]
            titles_list.append(t)
    
    return titles_list
        
    

In [10]:
from tqdm import tqdm

list_dics = list()

for npp in tqdm(list(npp_ts.npp)):
    dfnpp = pd.read_csv(npp + "-1815-1914.csv-metadata.csv", sep = "\t")

    dict_npp = dict()
    for year in list(set([l[0:4] for l in list(dfnpp.date)])):
        ltnppy = GetTitleNppYear(npp, int(year))
        dict_npp.update({"_".join([npp, str(year)]): ltnppy})
    list_dics.append(dict_npp)

  0%|                                                                                           | 0/21 [00:00<?, ?it/s]

ValueError: invalid literal for int() with base 10: 'date'